In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pyrealsense2 as rs
from pyntcloud import PyntCloud

In [11]:
pipeline = rs.pipeline()
config = rs.config()    
rs.config.enable_device_from_file(config, 'multicam1.bag')
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)   
#config.enable_stream(rs.stream.color, 640, 480, rs.format.z16, 60)  
#start streaming
pipeline.start(config)


In [12]:
#playback recording 
cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)
colorizer = rs.colorizer()

frame_list = []
try:
    while True:
        frames = pipeline.wait_for_frames()
        #depth frames
        depth_frame = frames.get_depth_frame()
        depth_frame_raw = np.asanyarray(depth_frame.get_data())
        depth_color_frame = colorizer.colorize(depth_frame)
        depth_color_image = np.asanyarray(depth_color_frame.get_data())
        
        #color frames
        color_frame = frames.get_color_frame()
        
        cv2.imshow("Depth Stream", depth_color_image)
        key = cv2.waitKey(1)
        if key == 27: #esc
            cv2.destroyAllWindows()
            break
            
        frame_count = depth_frame.get_frame_metadata(rs.frame_metadata_value.frame_counter)
        frame_list.append(frame_count)
        if frame_count == 5400:
            break
            
finally:
    cv2.destroyAllWindows()
    pipeline.stop()
            
        